In [1]:
import pandas as pd
import datetime
import time
import math

In [2]:
folder = '07_12_2018_09_51'
res_adapt = pd.read_csv(folder + '/adapt.csv')
res_adapt_eval = pd.read_csv(folder + '/adapt_eval.csv')
res_eval = pd.read_csv(folder + '/eval.csv')

In [3]:
res_adapt.drop(res_adapt[res_adapt.instance.str.contains('exact')].index, inplace=True)
res_eval.drop(res_eval[res_eval.instance.str.contains('exact')].index, inplace=True)
res_adapt_eval.drop(res_adapt_eval[res_adapt_eval.instance.str.contains('exact')].index, inplace=True)

In [4]:
res_eval['diff_p'] = (res_eval['pred_p'] - res_eval['real_p'])
res_eval['diff_p'] = res_eval['diff_p']*res_eval['diff_p']

In [5]:
res_eval.groupby('type')[['diff_p']].mean()

,diff_p
type,
CNN,5.606131e-01
CRNN,5.164956e-05
DNN,7.142669e-02
FeatureBased_Bayesian Ridge Regression,5.235106e-05
FeatureBased_Lasso Regression,5.235106e-05
FeatureBased_Linear Regression,5.235106e-05
FeatureBased_Passive Aggressive Regression,1.447790e-04
FeatureBased_SGD Regression,3.595260e+18
FeatureBased_SVM,5.235106e-05


In [6]:
res_eval.groupby('type')[['real_p','pred_p']].corr().iloc[0::2,-1]

type                                              
CNN                                         real_p    1.960262e-02
CRNN                                        real_p    1.184513e-01
DNN                                         real_p    5.546669e-02
FeatureBased_Bayesian Ridge Regression      real_p   -8.867088e-15
FeatureBased_Lasso Regression               real_p   -8.867088e-15
FeatureBased_Linear Regression              real_p   -8.867088e-15
FeatureBased_Passive Aggressive Regression  real_p             NaN
FeatureBased_SGD Regression                 real_p   -6.107902e-01
FeatureBased_SVM                            real_p   -8.794758e-15
FeatureBased_Theil-Sen Regression           real_p   -6.109572e-01
GRU                                         real_p   -1.661888e-02
Name: pred_p, dtype: float64

In [7]:
res_adapt['diff_p'] = res_adapt['new_p'] - res_adapt['old_p']
res_adapt['diff_r'] = res_adapt['new_r'] - res_adapt['old_r']
res_adapt['diff_f'] = res_adapt['new_f'] - res_adapt['old_f']

In [11]:
res_adapt.groupby('type').mean()

,Unnamed: 0,k,old_p,old_r,old_f,new_p,new_r,new_f,diff_p,diff_r,diff_f
type,,,,,,,,,,,
BPR,4972,0,0.005235,0.5,0.010323,0.005235,0.500000,0.010323,0.000000,0.000000,0.000000
CNN,4974,0,0.005235,0.5,0.010323,0.657810,0.614591,0.625985,0.652574,0.114591,0.615662
CRNN,4976,0,0.005235,0.5,0.010323,0.658077,0.614535,0.625856,0.652842,0.114535,0.615533
DNN,4975,0,0.005235,0.5,0.010323,0.658265,0.605640,0.619736,0.653030,0.105640,0.609413
GRU,4973,0,0.005235,0.5,0.010323,0.527742,0.524818,0.523565,0.522507,0.024818,0.513242
IR,4970,0,0.005235,0.5,0.010323,0.592200,0.532474,0.544576,0.586964,0.032474,0.534252
SVDpp,4971,0,0.005235,0.5,0.010323,0.502153,0.502680,0.501798,0.496918,0.002680,0.491475


In [9]:
res_adapt_eval['pred_e_max'] = res_adapt_eval.groupby(['instance','type'])['pred_e'].transform(max)
res_adapt_eval['k_max'] = res_adapt_eval.groupby(['instance','type'])['k'].transform(max)
clean_a_e = res_adapt_eval[(res_adapt_eval['pred_e_max'] == res_adapt_eval['pred_e']) &
               (res_adapt_eval['k_max'] == res_adapt_eval['k'])]

In [10]:
clean_a_e.groupby('type').mean()[['new_p', 'new_r', 'new_f']]

,new_p,new_r,new_f
type,,,
BPR_FEATBayesian Ridge Regression,0.005235,0.500000,0.010323
BPR_FEATLasso Regression,0.005235,0.500000,0.010323
BPR_FEATLinear Regression,0.005235,0.500000,0.010323
BPR_FEATPassive Aggressive Regression,0.005235,0.500000,0.010323
BPR_FEATSGD Regression,0.005235,0.500000,0.010323
BPR_FEATSVM,0.005235,0.500000,0.010323
BPR_FEATTheil-Sen Regression,0.005235,0.500000,0.010323
BPR_GRU,0.005235,0.500000,0.010323
CNN_CNN,0.657813,0.614593,0.625988


In [21]:
def summerize(data, file):
    idMax = data[data['type'] == data['type']]
    if 'pred_e' in data:
        idMax = data.groupby(['type'])['pred_e'].transform(max) == data['pred_e']
        pd.DataFrame(data[idMax].groupby(by='type').mean()).to_csv(file)
    else:
        pd.DataFrame(data.groupby(by='type').mean()).to_csv(file)

In [23]:
data = pd.read_csv('adapt.csv', low_memory=False)
summerize(data, 'a.csv')

ValueError: Must pass DataFrame with boolean values only